In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_density_weighted_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    # Initialize NearestNeighbors for density calculation
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(X_pool)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]

        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        # Calculate uncertainty: 1 - max(probability)
        uncertainties = 1 - np.max(probs_c, axis=1)  # shape: (n_samples_in_class,)

        # Calculate density: inverse of sum of distances to k-nearest neighbors
        distances = nn.kneighbors(X_c, n_neighbors=5, return_distance=True)[0]
        density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)

        # Combine uncertainty and density
        scores = uncertainties * density

        # Select top-k samples with highest scores
        sorted_idx = np.argsort(scores)[-n_select:]
        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = vstack(X_selected).toarray() if len(X_selected) > 0 else np.array([])
    y_selected = np.concatenate(y_selected) if len(y_selected) > 0 else np.array([])

    # Remaining data
    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=11)}")

    return X_rest, X_selected, y_rest, y_selected


# Example usage
X_pool = x_test_adv_0_1  # Your data
y_pool = y_label_0_1     # Your labels (0 to 10)

# Initial stratified sampling
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)
print(f"Initial labeled set size: {len(y_train_init)}")
print(f"Initial label distribution: {np.bincount(y_train_init, minlength=11)}")

# Train initial model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)

_, X_50, _, y_50 = classwise_density_weighted_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.5, random_state=42
)


Initial labeled set size: 102570
Initial label distribution: [75542   161   700  1743   377 10542    74   736 12435   172    88]


/tmp/ipykernel_541/4249287828.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_541/4249287828.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_541/4249287828.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_541/4249287828.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_541/4249287828.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_541/4249287828.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_541/4249287828.py:59: RuntimeWarning: divide by zero encountered in divide
  

Selected samples (fraction=0.5): 512867
Label distribution: [377713    807   3502   8717   1889  52710    370   3682  62176    861
    440]


In [6]:
X_50.shape

(512867, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt50 = DecisionTreeClassifier()
dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [60754   132  2115   971    67  4211   278 45063   296    84]
(60754, 56) (60754,)
Save 0 to baseline_Def1.npy
(132, 56) (132,)
Save 1 to baseline_Def2.npy
(2115, 56) (2115,)
Save 2 to baseline_Def3.npy
(971, 56) (971,)
Save 3 to baseline_Def4.npy
(67, 56) (67,)
Save 4 to baseline_Def5.npy
(4211, 56) (4211,)
Save 5 to baseline_Def6.npy
(278, 56) (278,)
Save 7 to baseline_Def8.npy
(45063, 56) (45063,)
Save 8 to baseline_Def9.npy
(296, 56) (296,)
Save 9 to baseline_Def10.npy
(84, 56) (84,)
Save 10 to baseline_Def11.npy
Execution Time: 0.369515 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [153637    106   1141   2992    133 147529     59  36267     44      5]
(153637, 56) (153637,)
Save 0 to BIM_Def1.npy
(106, 56) (106,)
Save 1 to BIM_Def2.npy
(1141, 56) (1141,)
Save 2 to BIM_Def3.npy
(2992, 56) (2992,)
Save 3 to BIM_Def4.npy
(133, 56) (133,)
Save 4 to BIM_Def5.npy
(147529, 56) (147529,)
Save 5 to BIM_Def6.npy
(59, 56) (59,)
Save 7 to BIM_Def

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [55276   174  1863  1021    98  4337   362 50450   289   101]
(55276, 56) (55276,)
Save 0 to baseline_Def1.npy
(174, 56) (174,)
Save 1 to baseline_Def2.npy
(1863, 56) (1863,)
Save 2 to baseline_Def3.npy
(1021, 56) (1021,)
Save 3 to baseline_Def4.npy
(98, 56) (98,)
Save 4 to baseline_Def5.npy
(4337, 56) (4337,)
Save 5 to baseline_Def6.npy
(362, 56) (362,)
Save 7 to baseline_Def8.npy
(50450, 56) (50450,)
Save 8 to baseline_Def9.npy
(289, 56) (289,)
Save 9 to baseline_Def10.npy
(101, 56) (101,)
Save 10 to baseline_Def11.npy
Execution Time: 2.523455 seconds
(341913,)
[0 2 3 4 5 8 9] [146900    566   1182      2 180276  12975     12]
(146900, 56) (146900,)
Save 0 to BIM_Def1.npy
(566, 56) (566,)
Save 2 to BIM_Def3.npy
(1182, 56) (1182,)
Save 3 to BIM_Def4.npy
(2, 56) (2,)
Save 4 to BIM_Def5.npy
(180276, 56) (180276,)
Save 5 to BIM_Def6.npy
(12975, 56) (12975,)
Save 8 to BIM_Def9.npy
(12, 56) (12,)
Save 9 to BIM_Def10.npy
Execution Time: 3.067419 sec

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [64286   877  1841   440   212 10775   414 34795   245    86]
(64286, 56) (64286,)
Save 0 to baseline_Def1.npy
(877, 56) (877,)
Save 1 to baseline_Def2.npy
(1841, 56) (1841,)
Save 2 to baseline_Def3.npy
(440, 56) (440,)
Save 3 to baseline_Def4.npy
(212, 56) (212,)
Save 4 to baseline_Def5.npy
(10775, 56) (10775,)
Save 5 to baseline_Def6.npy
(414, 56) (414,)
Save 7 to baseline_Def8.npy
(34795, 56) (34795,)
Save 8 to baseline_Def9.npy
(245, 56) (245,)
Save 9 to baseline_Def10.npy
(86, 56) (86,)
Save 10 to baseline_Def11.npy
Execution Time: 0.099593 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [205070   1571   1994   7598    154  94641   7690  22916    239     40]
(205070, 56) (205070,)
Save 0 to BIM_Def1.npy
(1571, 56) (1571,)
Save 1 to BIM_Def2.npy
(1994, 56) (1994,)
Save 2 to BIM_Def3.npy
(7598, 56) (7598,)
Save 3 to BIM_Def4.npy
(154, 56) (154,)
Save 4 to BIM_Def5.npy
(94641, 56) (94641,)
Save 5 to BIM_Def6.npy
(7690, 56) (7690,)
Save 7 to

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_DensityWeighted/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ActiveLearning_DensityWeighted.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,60754,132,2115,971,67,4211,0,278,45063,296,84,113971
1,XGB50,153637,106,1141,2992,133,147529,0,59,36267,44,5,341913
2,XGB50,134302,107,1156,2870,25,167341,0,265,35817,12,18,341913
3,XGB50,153637,106,1141,2992,133,147529,0,59,36267,44,5,341913
4,XGB50,78376,0,1411,184,92,246304,746,13805,653,321,21,341913
5,XGB50,165322,277,4055,2622,255,154960,0,652,13121,412,237,341913
6,XGB50,167370,388,6150,3401,196,11992,0,831,150529,803,253,341913
7,XGB50,272339,19,387,725,4,21137,0,133,46252,648,269,341913
8,XGB50,133224,94,704,1572,32,180978,0,9,25252,41,7,341913
9,XGB50,133547,83,1162,2720,67,168813,0,37,35435,21,28,341913
